In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import sys

In [5]:
path_data_source = 'smart_heating_dataset/'
path_data_extracted = "extracted_dataset/"
data_files = os.listdir(path_data_source)
try:
    os.mkdir(path_data_extracted)
except FileExistsError:
    pass

nums = sorted(list(set([i[:6] for i in data_files if i != "weather.json"])))
params = sorted(list(set([i[6:-5] for i in data_files if i != "weather.json"])))

weather = pd.read_json(path_data_source+"weather.json", typ='frame', date_unit='s')
weather.set_index(pd.to_datetime(weather['time'], unit='ms'), verify_integrity=False, inplace=True)
weather.drop(['time'], inplace=True, axis=1)

t1, t2 = pd.Timestamp('2017-11-30 21:00:00').timestamp(), pd.Timestamp('2018-03-15 03:00:00').timestamp()
times = pd.to_datetime(np.linspace(t1+3*60*60, t2+3*60*60, (t2-t1)//60+1), unit='s')

/home/selkie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.


In [6]:
for num in [i for i in nums if not (i[:-1]+'.csv' in os.listdir(path_data_extracted))]:
    print(num)
    tube = pd.DataFrame() 
    for param in params:
        p = pd.read_json(path_data_source+num+param+".json", typ='series')
        pf = p[times]
        p = p[times].dropna()
        indexes = p.index
        first, last = p[0], p[-1]
        prev, last_id = indexes[0], indexes[-1]
        iter_ids = iter(indexes[1:])
        pf[:prev] = first
        pf[last_id:] = last

        for curr in tqdm(p.index[1:]):
            pf[prev:curr] = np.round(np.linspace(p[prev], p[curr], len(pf[prev:curr])), 4)
            prev = curr
        tube[param] = pf[180:-180]
    tube.to_csv(path_data_extracted+num[:-1]+'.csv', index_label='timestamp')

In [9]:
weather_new = pd.DataFrame()
for param in weather.columns:
    p = weather[param]
    pf = p[times]
    p = p[times].dropna()
    indexes = p.index
    first = p[0]
    prev, last_id = indexes[0], indexes[-1]
    iter_ids = iter(indexes[1:])
    pf[:prev] = first

    for curr in tqdm(p.index[1:]):
        pf[prev:curr] = np.round(np.linspace(p[prev], p[curr], len(pf[prev:curr])), 1)
        prev = curr
    weather_new[param] = pf[180:-180]
weather_new.to_csv(path_data_extracted+'weather.csv', index_label='timestamp')

100%|██████████| 2456/2456 [00:00<00:00, 3009.32it/s]
